In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from pathlib import Path
from scipy.optimize import linear_sum_assignment

In [2]:
OUTPUT_DIR = Path("/mnt/jwh83-data/Confetti/output/")

GROUND_TRUTHS = {
    "nature": ("nature_B004_registered_IDmatched.h5ad", "Cell Type"),
    "hubmap": ("HuBMAP_Yang_annotate.h5ad", "cell_type_update"),
}

METHODS = {
    "mesmer":      ("Channel_feature/Merged/mesmer_leiden_prelim.h5ad", "leiden"),
    "cellpose":    ("Channel_feature/Merged/cellpose_old_update3_leiden_prelim.h5ad", "leiden"),
    "cellposeSAM": ("Channel_feature/Merged/cellposeSAM_leiden_prelim.h5ad", "leiden"),
    "microSAM":    ("Channel_feature/Merged/microSAM_leiden_prelim.h5ad", "leiden"),
    "cellSAM":     ("Channel_feature/Merged/cellSAM_2_update_leiden_prelim.h5ad", "leiden"),
    "instanseg":   ("Channel_feature/Merged/instanseg_leiden_prelim.h5ad", "leiden"),
}

EPS = 1e-12

In [3]:
def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else np.asarray(X)


def ordered_intersection(a, b):
    bset = set(b)
    return [x for x in a if x in bset]


def compute_centroids(adata, cluster_key, features):
    X = to_dense(adata[:, features].X)
    df = pd.DataFrame(X, columns=features)
    df["cluster"] = adata.obs[cluster_key].astype(str).values
    centroids = df.groupby("cluster", observed=True).mean()
    sizes = df.groupby("cluster", observed=True).size()
    return centroids, sizes


def cosine_similarity(gt, pred):
    A = gt.values
    B = pred.values
    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + EPS)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + EPS)
    return pd.DataFrame(A @ B.T, index=gt.index, columns=pred.index)


def summarize_many_to_one(sim, gt_sizes):
    best = sim.max(axis=1)
    w = gt_sizes.reindex(sim.index).fillna(0).values
    return {
        "m2o_macro_mean": best.mean(),
        "m2o_weighted_mean": np.average(best.values, weights=w) if w.sum() > 0 else np.nan,
    }


def summarize_one_to_one(sim, gt_sizes):
    cost = -sim.values
    r, c = linear_sum_assignment(cost)
    matched = sim.values[r, c]
    sizes = gt_sizes.reindex(sim.index[r]).fillna(0).values
    return {
        "o2o_macro_mean": matched.mean() if matched.size else np.nan,
        "o2o_weighted_mean": np.average(matched, weights=sizes) if sizes.sum() > 0 else np.nan,
        "n_matched": matched.size,
    }


In [4]:
gt_adatas = {
    name: ad.read_h5ad(OUTPUT_DIR / path)
    for name, (path, _) in GROUND_TRUTHS.items()
}
method_adatas = {
    name: ad.read_h5ad(OUTPUT_DIR / path)
    for name, (path, _) in METHODS.items()
}

rows = []

/home/john-hickey/anaconda3/envs/cellpose/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
for gt_name, (_, gt_key) in GROUND_TRUTHS.items():
    adata_gt = gt_adatas[gt_name]

    for method_name, (_, method_key) in METHODS.items():
        adata_m = method_adatas[method_name]

        common_features = ordered_intersection(
            adata_gt.var_names, adata_m.var_names
        )
        if len(common_features) < 2:
            continue

        gt_centroids, gt_sizes = compute_centroids(
            adata_gt, gt_key, common_features
        )
        m_centroids, _ = compute_centroids(
            adata_m, method_key, common_features
        )

        sim = cosine_similarity(gt_centroids, m_centroids)

        m2o = summarize_many_to_one(sim, gt_sizes)
        o2o = summarize_one_to_one(sim, gt_sizes)

        row = {
            "gt": gt_name,
            "method": method_name,
            "n_features": len(common_features),
            "n_gt_clusters": gt_centroids.shape[0],
            "n_pred_clusters": m_centroids.shape[0],
            **m2o,
            **o2o,
        }
        rows.append(row)

        print(f"\nGT: {gt_name} | Method: {method_name}")
        print(f"  Features used: {len(common_features)}")
        print(f"  GT clusters: {gt_centroids.shape[0]}")
        print(f"  Pred clusters: {m_centroids.shape[0]}")
        print(f"  M2O mean cosine: {m2o['m2o_macro_mean']:.3f}")
        print(f"  O2O mean cosine: {o2o['o2o_macro_mean']:.3f}")

summary_df = pd.DataFrame(rows)

print("\n=== Overall summary ===")
print(
    summary_df.sort_values(
        ["gt", "m2o_macro_mean"], ascending=[True, False]
    ).to_string(index=False)
)


GT: nature | Method: mesmer
  Features used: 41
  GT clusters: 25
  Pred clusters: 41
  M2O mean cosine: 0.821
  O2O mean cosine: 0.819

GT: nature | Method: cellpose
  Features used: 41
  GT clusters: 25
  Pred clusters: 38
  M2O mean cosine: 0.827
  O2O mean cosine: 0.827

GT: nature | Method: cellposeSAM
  Features used: 41
  GT clusters: 25
  Pred clusters: 43
  M2O mean cosine: 0.836
  O2O mean cosine: 0.833

GT: nature | Method: microSAM
  Features used: 41
  GT clusters: 25
  Pred clusters: 45
  M2O mean cosine: 0.824
  O2O mean cosine: 0.824

GT: nature | Method: cellSAM
  Features used: 41
  GT clusters: 25
  Pred clusters: 39
  M2O mean cosine: 0.825
  O2O mean cosine: 0.817

GT: nature | Method: instanseg
  Features used: 41
  GT clusters: 25
  Pred clusters: 40
  M2O mean cosine: 0.838
  O2O mean cosine: 0.836

GT: hubmap | Method: mesmer
  Features used: 45
  GT clusters: 28
  Pred clusters: 41
  M2O mean cosine: 0.858
  O2O mean cosine: 0.843

GT: hubmap | Method: cellpo

In [6]:
from scipy.stats import friedmanchisquare, wilcoxon
from statsmodels.stats.multitest import multipletests

In [7]:
per_cluster_rows = []

for gt_name, (_, gt_key) in GROUND_TRUTHS.items():
    adata_gt = gt_adatas[gt_name]

    for method_name, (_, method_key) in METHODS.items():
        adata_m = method_adatas[method_name]

        common_features = ordered_intersection(adata_gt.var_names, adata_m.var_names)
        if len(common_features) < 2:
            continue

        gt_centroids, gt_sizes = compute_centroids(adata_gt, gt_key, common_features)
        m_centroids, _ = compute_centroids(adata_m, method_key, common_features)

        sim = cosine_similarity(gt_centroids, m_centroids)

        best_pred = sim.idxmax(axis=1)
        best_sim  = sim.max(axis=1)

        tmp = pd.DataFrame({
            "gt": gt_name,
            "method": method_name,
            "gt_cluster": sim.index.astype(str),
            "gt_cluster_size": gt_sizes.reindex(sim.index).fillna(0).astype(int).values,
            "best_pred_cluster": best_pred.values.astype(str),
            "best_cosine_m2o": best_sim.values.astype(float),
        })
        per_cluster_rows.append(tmp)

per_cluster_df = pd.concat(per_cluster_rows, ignore_index=True)

print("\n=== Per-GT-cluster table (head) ===")
print(per_cluster_df.head().to_string(index=False))


=== Per-GT-cluster table (head) ===
    gt method       gt_cluster  gt_cluster_size best_pred_cluster  best_cosine_m2o
nature mesmer                B             3495                40         0.806192
nature mesmer      CD4+ T cell            11331                33         0.868043
nature mesmer CD57+ Enterocyte              102                 8         0.347470
nature mesmer CD66+ Enterocyte            12379                27         0.824146
nature mesmer      CD7+ Immune             1584                36         0.943774


In [8]:
def stats_for_gt(per_cluster_df, gt_name, metric_col="best_cosine_m2o"):
    sub = per_cluster_df[per_cluster_df["gt"] == gt_name].copy()
    methods = sorted(sub["method"].unique())
    clusters = sorted(sub["gt_cluster"].unique())

    # pivot: rows=gt_cluster blocks, cols=methods treatments
    wide = sub.pivot_table(index="gt_cluster", columns="method", values=metric_col, aggfunc="mean")

    # keep only clusters with complete data across methods
    wide = wide.dropna(axis=0, how="any")
    wide = wide[methods]  # enforce consistent column order

    print(f"\n=== {gt_name}: n_blocks={wide.shape[0]} | n_methods={wide.shape[1]} ===")

    # Friedman test
    fried_stat, fried_p = friedmanchisquare(*[wide[m].values for m in methods])
    print(f"Friedman chi2={fried_stat:.3f}, p={fried_p:.3g}")

    # Pairwise Wilcoxon + Holm
    pairs = []
    pvals = []
    effects = []
    dominance = []

    for i in range(len(methods)):
        for j in range(i + 1, len(methods)):
            a, b = methods[i], methods[j]
            x = wide[a].values
            y = wide[b].values

            d = x - y
            # Wilcoxon signed-rank (paired)
            try:
                w_stat, p = wilcoxon(d, zero_method="wilcox", alternative="two-sided")
            except ValueError:
                # all differences zero or insufficient variation
                p = 1.0
                w_stat = np.nan

            pairs.append((a, b))
            pvals.append(p)

            # effect summaries
            effects.append(np.median(d))
            dominance.append(np.mean(d > 0))  # fraction blocks where a > b

    reject, p_adj, _, _ = multipletests(pvals, method="holm")

    posthoc = pd.DataFrame({
        "gt": gt_name,
        "A": [a for a, b in pairs],
        "B": [b for a, b in pairs],
        "median_diff(A-B)": effects,
        "dominance_P(A>B)": dominance,
        "p_value": pvals,
        "p_adj_holm": p_adj,
        "reject_0.05": reject,
    }).sort_values(["p_adj_holm", "p_value"])

    print("\nPost-hoc (top 15 by Holm-adjusted p):")
    print(posthoc.head(15).to_string(index=False))

    # Method ranking by median per-block score
    ranks = (
        wide.median(axis=0)
        .sort_values(ascending=False)
        .reset_index()
        .rename(columns={"index": "method", 0: "median_score"})
    )
    print("\nMethod ranking (by median per-GT-cluster best_cosine_m2o):")
    print(ranks.to_string(index=False))

    best_two = ranks["method"].iloc[:2].tolist()
    print(f"\nBest two methods for {gt_name}: {best_two}")

    return {"friedman_p": fried_p, "ranks": ranks, "posthoc": posthoc, "wide": wide}


In [9]:
gt_results = {}
for gt_name in sorted(per_cluster_df["gt"].unique()):
    gt_results[gt_name] = stats_for_gt(per_cluster_df, gt_name, metric_col="best_cosine_m2o")

pooled = per_cluster_df.copy()
pooled["block"] = pooled["gt"].astype(str) + "||" + pooled["gt_cluster"].astype(str)
wide_pooled = pooled.pivot_table(index="block", columns="method", values="best_cosine_m2o", aggfunc="mean").dropna(axis=0, how="any")
methods = sorted(wide_pooled.columns.tolist())
wide_pooled = wide_pooled[methods]

pooled_ranks = (
    wide_pooled.median(axis=0)
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={"index": "method", 0: "median_score"})
)

print("\n=== Pooled across GTs: method ranking by median per-block score ===")
print(pooled_ranks.to_string(index=False))
print(f"\nBest two methods (pooled): {pooled_ranks['method'].iloc[:2].tolist()}")


=== hubmap: n_blocks=28 | n_methods=6 ===
Friedman chi2=6.939, p=0.225

Post-hoc (top 15 by Holm-adjusted p):
    gt           A           B  median_diff(A-B)  dominance_P(A>B)  p_value  p_adj_holm  reject_0.05
hubmap   instanseg      mesmer         -0.008914          0.321429 0.099280         1.0        False
hubmap cellposeSAM   instanseg          0.004494          0.607143 0.163799         1.0        False
hubmap    cellpose      mesmer         -0.003225          0.357143 0.284213         1.0        False
hubmap     cellSAM cellposeSAM         -0.003007          0.392857 0.294598         1.0        False
hubmap     cellSAM      mesmer         -0.006837          0.392857 0.362001         1.0        False
hubmap cellposeSAM      mesmer          0.003633          0.571429 0.362001         1.0        False
hubmap    cellpose cellposeSAM         -0.002019          0.464286 0.424753         1.0        False
hubmap cellposeSAM    microSAM         -0.001510          0.500000 0.465176      